# Simulation of an (M,L) Inventory System
## Describing the System

An (M, L) inventory system has a maximum inventory level of 50 and a reorder point of 30. The inventory is checked at the end of each month. If current inventory levels plus inventory on order is less than 30, a new order is placed. If current inventory plus inventory on order is greater than or equal to 30, no order is placed at the end of the month. The procurement quantity Q in the order is defined by the maximum inventory level (M) minus the current inventory plus inventory on order (I). Backordering is allowed, but it comes at a cost of $4 per item short per month.

Lead time of new orders varies according to a uniform distribution on the interval [0.25, 1.25] months. Time between customer demands is exponentially distributed with a mean of 1/15 month. The size of the demands follows this empirical distribution:
*   1/2 probability of 1 demand
*   1/4 probability of 2 demand
*   1/8 probability of 3 demand
*   1/8 probability of 4 demand

The starting inventory in the inventory system is 50 units, with no orders outstanding.
<br/>

## Problem Description

The objective of this simulation is to solve for the mean monthly cost of the inventory system. The costs associated with the system are as follows:
*   Backlog: \$4 per item short per month
*   Holding: \$1 per unit in inventory per month
*   Ordering: \$60 per order
*   Item: \$5 per item ordered

Ten replications of the simulation will be generated, and then the long-run mean monthly cost will be estimated by averaging the results across replications. In addition, a 90% confidence interval will be constructed.

If the results from the ten replications lead to a confidence interval wider than \\$5, the total number of additional replications needed to shrink the confidence interval to \$5 will be estimated.


## Building the Simulation Model
To build this simulation model, numpy is primarily used. The random function is also needed to create stochastic lead time and customer demand. Scipy is used for statistical analysis after the simulation is completed. These libraries are imported below.

In [1]:
#import libraries
import numpy as np
import pandas as pd
from scipy import stats
import random

The simulation is built using for loops and while loops. First, the time is intialized to 0, inventory level is set to 50, and all necessary vectors are initialized so they are stored globally.

The simulation moves forward as new customer orders arrive. Each time a customer order arrives, inventory is subtracted from demand, time advances based on the interarrival time, and backlog is documented if there is no remaining inventory. At the end of a month (when the variable time crosses an integer value), inventory levels are checked. If inventory plus outstanding orders is less than 30, a new order is generated as well as order lead time. The order quantity, backlog for the month, and average inventory levels are stored at the end of each month, and then the loop repeats for 112 months. 

After the 112 months are completed, the average monthly cost is found (using a 12 month initialization period) and stored. This entire process is looped 10 times, so the final result is a 'cost' vector containing the long-run mean monthly cost of each of the 10 replications. 

In [58]:
#run 10 simulations and store results in cost vector
cost = np.array([])
for i in range(10):
    time = 0
    inv_level = 50
    b = np.zeros(112)
    orders = np.zeros(112)
    inventory = np.zeros(112)
    reorder_quantity = 0
    arrive_time_of_order = 0
    monthly_cost = np.zeros(112)
    for i in range(1, 112):
        while time < i:
            inv_average = np.array([])
            interarrival = -1/15 * np.log(random.random())
            time += interarrival
            x = random.random()
            if x < 0.5:
                demand = 1
            elif x < 0.75:
                demand = 2
            elif x < 0.875:
                demand = 3
            else:
                demand = 4
            inv_level = inv_level - demand
            if arrive_time_of_order > time:
                inv_level += reorder_quantity
                reorder_quantity = 0
                arrive_time_of_order = 0
            if inv_level < 0:
                b[i] += demand
            inv_average = np.append(inv_average, inv_level)
        if inv_level < 30:
            reorder_quantity = 50 - inv_level
            arrive_time_of_order = time + random.random()+0.25
        orders[i] = reorder_quantity
        inventory[i] = sum(inv_average)/len(inv_average)
    for i in range(len(monthly_cost)):
        if orders[i] > 0:
            monthly_cost[i] = orders[i] * 5 + 60 + inventory[i] * 1 + b[i] * 4
        else:
            monthly_cost[i] = inventory[i] * 1 + b[i] * 4
    average_long_run_monthly_cost = \
    sum(monthly_cost[12:])/len(monthly_cost[12:])
    cost = np.append(cost, average_long_run_monthly_cost)

## Estimating the Monthly Cost
After the ten replications are run, the point estimate for the long-run mean monthly cost is found by taking the average value across the ten replications.

In [59]:
#solve for point estimate of cost
point_estimate = sum(cost)/len(cost)
print('$',point_estimate.round(2))

$ 216.76


The 90% confidence is found using the t-distribution. The t-statistic is found using scipy, and the standard deviation is found using numpy.

In [69]:
#solve for confidence interval
var_s = 0
for i in range(len(cost)):
    var_s += (cost[i] - point_estimate)**2
var_s /= (len(cost)-1)
std_dev_s = np.sqrt(var_s)
t_stat = stats.t.ppf(1-0.05, 10-1)
half_width = t_stat * std_dev_s / np.sqrt(10)
upper_bound = point_estimate + half_width
lower_bound = point_estimate - half_width
print('Confidence Interval: ({:.2f},{:.2f})'.format(lower_bound, upper_bound))

Confidence Interval: (213.78,219.73)


## Estimating Number of Additional Replications Needed


In [70]:
R0 = 10
error = 5
z_value = 1.645
R_initial_estimate = ((z_value * std_dev_s)/error) ** 2
print(R_initial_estimate)

2.8543459585511126


In [74]:
t_stat_3rep = stats.t.ppf(1-0.05, 3-1)
R = ((t_stat_3rep * std_dev_s)/error) ** 2
print(R)

8.993654905274191


In [76]:
t_stat_4rep = stats.t.ppf(1-0.05, 4-1)
R = ((t_stat_4rep * std_dev_s)/error) ** 2
print(R)

5.84188237623346


In [78]:
t_stat_5rep = stats.t.ppf(1-0.05, 5-1)
R = ((t_stat_5rep * std_dev_s)/error) ** 2
print(R)

4.793875842795313


In [79]:
if R-R0 > 0:
    additional_rep = R-R0
else:
    additional_rep = 0
print(additional_rep)

0


In [85]:
R_guess = 3
R = 4
while R > R_guess:
    t_stat = stats.t.ppf(1-0.05, R_guess-1)
    R = ((t_stat * std_dev_s)/error) ** 2
    if R > R_guess:
        R_guess += 1
print(R_guess)

5
